In [2]:
import set_cwd
import pandas_settings
from src import PathOrganizer
from src import Model_Exploration_Tool as es
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)

lmur_pathorg = PathOrganizer()
model_dir = lmur_pathorg.get_model_version_path(mo_name="LigilactobacillusMurinus")
model = lmur_pathorg.load_model(model_dir)

closed_uptake = [
    "EX_cpd00007_e0",  # Oxygen
]

essential = [
    "EX_cpd10516_e0",  # fe3
]

c_sources = [
    "EX_cpd00027_e0",  # glucose
    "EX_cpd01200_e0",  # palatinose
    "EX_cpd00138_e0",  # mannose
    "EX_cpd00076_e0",  # sucrose
    "EX_cpd00082_e0",  # fructose
    "EX_cpd00020_e0",  # pyruvate
    "EX_cpd00751_e0",  # fucose
    "EX_cpd00280_e0",  # galacturonic acid
    "EX_cpd00020_e0",  # pyruvate
]

kwoji_updated = {
    "EX_cpd00009_e0": (-60, 1000),
    "EX_cpd00205_e0": (-95, 1000),
    "EX_cpd00067_e0": (-92, 1000),
    "EX_cpd00254_e0": (-25, 1000),
    "EX_cpd00048_e0": (-25.56, 1000),
    "EX_cpd00001_e0": (-180.21, 1000),
    "EX_cpd00030_e0": (-0.25, 1000),
    "EX_cpd00013_e0": (-10, 1000),
    "EX_cpd00099_e0": (-27.02, 1000),
    "EX_cpd10515_e0": (-0.2, 1000),
    "EX_cpd00971_e0": (-52, 1000),
    "EX_cpd00011_e0": (-2, 1000),
    "EX_cpd00149_e0": (-0.01, 1000),
    "EX_cpd00063_e0": (-1, 1000),
    "EX_cpd00034_e0": (-0.1, 1000),
    "EX_cpd00058_e0": (-0.01, 1000),
    "EX_cpd00029_e0": (-50, 1000),
    "EX_cpd00028_e0": (-0.05, 1000),
    "EX_cpd00322_e0": (-2, 1000),
    "EX_cpd00060_e0": (-1, 1000),
    "EX_cpd00066_e0": (-1, 1000),
    "EX_cpd00119_e0": (-2.5, 1000),
    "EX_cpd00053_e0": (-2, 1000),
    "EX_cpd00023_e0": (-2, 1000),
    "EX_cpd00132_e0": (-2, 1000),
    "EX_cpd00041_e0": (-2, 1000),
    "EX_cpd00069_e0": (-1, 1000),
    "EX_cpd00065_e0": (-1, 1000),
    "EX_cpd00035_e0": (-1, 1000),
    "EX_cpd00156_e0": (-1, 1000),
    "EX_cpd00051_e0": (-1, 1000),
    "EX_cpd00129_e0": (-1, 1000),
    "EX_cpd00033_e0": (-1, 1000),
    "EX_cpd00039_e0": (-1, 1000),
    "EX_cpd00161_e0": (-1, 1000),
    "EX_cpd00107_e0": (-1, 1000),
    "EX_cpd00054_e0": (-1, 1000),
    "EX_cpd00084_e0": (-2, 1000),
    "EX_cpd00311_e0": (-0.1, 1000),
    "EX_cpd00182_e0": (-0.1, 1000),
    "EX_cpd01217_e0": (-0.1, 1000),
    "EX_cpd00184_e0": (-0.1, 1000),
    "EX_cpd00092_e0": (-0.1, 1000),
    "EX_cpd00218_e0": (-0.01, 1000),
    "EX_cpd00305_e0": (-0.01, 1000),
    "EX_cpd00644_e0": (-0.01, 1000),
    "EX_cpd00220_e0": (-0.01, 1000),
    "EX_cpd00393_e0": (-0.01, 1000),
    "EX_cpd00104_e0": (-0.005, 1000),
    "EX_cpd11606_e0": (-0.005, 1000),
    "EX_cpd00027_e0": (-50, 1000)
}

Available Model Versions:
1) 0.0.0.sbml
2) 1.1.1.sbml
3) 1.1.2.sbml
4) 1.1.3.sbml
5) 1.1.4.sbml
6) 1.1.5.sbml
7) 2.1.1.sbml
8) 2.2.1.sbml
9) 2.2.2.sbml
10) 2.2.3.sbml
11) 2.2.4.sbml
12) 2.2.5.sbml
13) 2.2.6.sbml
14) 2.2.7.sbml


In [3]:
exploration = es(model=model)
exploration.set_media(medium=kwoji_updated, essential=essential, closed=closed_uptake)

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
c:\Users\Adrianchi\AppData\Local\Programs\Python\Python310\lib\site-packages\cobra\medium\boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return most[0]


In [4]:
test = model.summary(fva=0.95).to_frame()

In [5]:
# test[["flux", "minimum", "maximum"]] != 0
ok = test[test["flux"]== 0].copy()

mimi = ok[ok["minimum"]!=0].copy()

mama = ok[ok["maximum"]!=0].copy()

In [7]:
mama

,reaction,metabolite,factor,flux,minimum,maximum
EX_cpd00067_e0,EX_cpd00067_e0,cpd00067_e0,-1.0,0.0,-156.08442,66.37283
EX_cpd11606_e0,EX_cpd11606_e0,cpd11606_e0,-1.0,0.0,0.00000,0.00500


In [8]:
mimi

,reaction,metabolite,factor,flux,minimum,maximum
DM_cpd01042_c0,DM_cpd01042_c0,cpd01042_c0,-1.0,0.0,-0.00500,0.00000
EX_cpd00010_e0,EX_cpd00010_e0,cpd00010_e0,-1.0,0.0,-0.00050,0.00000
EX_cpd00017_e0,EX_cpd00017_e0,cpd00017_e0,-1.0,0.0,-0.85989,0.00000
EX_cpd00020_e0,EX_cpd00020_e0,cpd00020_e0,-1.0,0.0,-105.45690,0.00000
EX_cpd00036_e0,EX_cpd00036_e0,cpd00036_e0,-1.0,0.0,-3.67926,0.00000
EX_cpd00067_e0,EX_cpd00067_e0,cpd00067_e0,-1.0,0.0,-156.08442,66.37283
EX_cpd00071_e0,EX_cpd00071_e0,cpd00071_e0,-1.0,0.0,-153.56312,0.00000
EX_cpd00080_e0,EX_cpd00080_e0,cpd00080_e0,-1.0,0.0,-58.76629,0.00000
EX_cpd00100_e0,EX_cpd00100_e0,cpd00100_e0,-1.0,0.0,-85.41809,0.00000
EX_cpd00105_e0,EX_cpd00105_e0,cpd00105_e0,-1.0,0.0,-49.03965,0.00000


In [6]:
model.summary(fva=0.95)

Metabolite,Reaction,Flux,Range,C-Number,C-Flux
cpd00009_e0,EX_cpd00009_e0,1.299,[1.234; 60],0,0.00%
cpd00023_e0,EX_cpd00023_e0,0.1335,[-1.394; 2],5,1.55%
cpd00027_e0,EX_cpd00027_e0,2.264,[0.9603; 50],6,31.60%
cpd00028_e0,EX_cpd00028_e0,0.005,[-0.1083; 0.005],34,0.40%
cpd00030_e0,EX_cpd00030_e0,0.007061,[0.006708; 0.007061],0,0.00%
cpd00033_e0,EX_cpd00033_e0,0.4948,[0.09587; 1],2,2.30%
cpd00034_e0,EX_cpd00034_e0,0.007061,[0.006708; 0.007061],0,0.00%
cpd00035_e0,EX_cpd00035_e0,0.5203,[-24.04; 1],3,3.63%
cpd00039_e0,EX_cpd00039_e0,0.3636,[-3.334; 0.3636],6,5.08%
cpd00041_e0,EX_cpd00041_e0,0.1763,[-1.679; 2],4,1.64%


In [13]:
model.metabolites.cpd00137_c0.summary()

Percent,Flux,Reaction,Definition
Percent,Flux,Reaction,Definition


In [14]:
model.metabolites.get_by_id("cpd00137_c0")

Metabolite identifier,cpd00137_c0
Name,Citrate-c0
Memory address,0x2b1b669f910
Formula,C6H5O7
Compartment,c0
In 2 reaction(s),"rxn12371_c0, rxn05211_c0"


In [16]:
model.reactions.get_by_id("rxn05211_c0")

Reaction identifier,rxn05211_c0
Name,"Transport of citrate, extracellular"
Memory address,0x2b1b6ebb670
Stoichiometry,3.0 cpd00067_e0 + cpd00137_e0 <=> 3.0 cpd00067_c0 + cpd00137_c0 3.0 H+-e0 + Citrate-e0 <=> 3.0 H+-c0 + Citrate-c0
GPR,gnl_DSMZ_unitig_0_quiver_consensus_311618_312865
Lower bound,-1000.0
Upper bound,1000.0
